<a href="https://colab.research.google.com/github/Eieiz007/NLP_learn/blob/main/Machine_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install library

In [ ]:
!pip install -U transformers huggingface_hub

In [ ]:
!pip install datasets==1.17.0
!pip install accelerate
!apt install git-lfs
!pip install sacrebleu==2.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.3/306.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Login

In [ ]:
!git config --global user.email "XXX"
!git config --global user.name "X"


```
```



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Import data

In [ ]:
from datasets import load_dataset, load_metric , DatasetDict
dataset = load_dataset("airesearch/scb_mt_enth_2020",'enth')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Downloading:   0%|          | 0.00/4.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset scb_mt_enth2020 downloaded and prepared to /root/.cache/huggingface/datasets/airesearch___scb_mt_enth2020/enth/1.0.0/3bb5a43042a84446b254e215a8b1604b454ad56ada3370309a6f6a3b480c86a8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 801402
    })
    validation: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 100173
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 100177
    })
})

In [ ]:
train_wiki = dataset["train"].filter(lambda x: x["subdataset"] == "wikipedia_corpus")
val_wiki = dataset["validation"].filter(lambda x: x["subdataset"] == "wikipedia")
test_wiki = dataset["test"].filter(lambda x: x["subdataset"] == "wikipedia")

wikipedia_dataset = DatasetDict({
    "train": train_wiki,
    "validation": val_wiki,
    "test": test_wiki
})

  0%|          | 0/802 [00:00<?, ?ba/s]

  0%|          | 0/101 [00:00<?, ?ba/s]

  0%|          | 0/101 [00:00<?, ?ba/s]

In [ ]:
wikipedia_dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 26163
    })
    validation: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 3796
    })
    test: Dataset({
        features: ['translation', 'subdataset'],
        num_rows: 3797
    })
})

In [ ]:
wikipedia_dataset['train']['translation'][1:2]

[{'en': 'Palestine competed at the 2004 Summer Olympics in Athens, Greece, from 13 to 29 August 2004.',
  'th': 'ปาเลสไตน์ในโอลิมปิกฤดูร้อน 2004 ปาเลสไตน์ เข้าร่วมแข่งขันกีฬาโอลิมปิกฤดูร้อนครั้งที่ 28 ค.ศ. 2004 (พ.ศ. 2547) ณ กรุงเอเธนส์ ประเทศกรีซระหว่างวันที่ 13 – 29 สิงหาคม พ.ศ. 2547'}]

# Data preprocessing

เนื่องจากเป้าหมาย thai -> eng แต่ datasets eng -> thai    ดังนั้นจะต้องสลับ ตำแหน่งของ datasets     

In [ ]:
wikipedia_dataset['train']['translation'][6]

{'en': 'FIFA 15 is an association football simulation video game developed by EA Canada and published by Electronic Arts.',
 'th': 'ฟีฟ่า 15 ฟีฟ่า 15 () เป็นวิดีโอเกมจําลองฟุตบอล ที่ได้รับการพัฒนาโดยอีเอ แคนาดา และได้รับการเผยแพร่โดยอิเล็กทรอนิก อาตส์'}

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-th-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="tf")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/810k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.40M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
def tokenize_function(examples):
    inputs = tokenizer([example['th'] for example in examples['translation']], padding=True, truncation=True)
    targets = tokenizer([example['en'] for example in examples['translation']], padding=True, truncation=True)
    #512 max_target_length
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids']
    }

tokenized_datasets = wikipedia_dataset.map(lambda x: tokenize_function(x), batched=True)

print(tokenized_datasets['train'][0])

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_ids': [12, 38724, 175, 27180, 13605, 10869, 0, 62306, 62306, 62306, 62306, 62306, 62306, 6

# Test token

In [ ]:
text = "คนไทยบินได้"
tokens = tokenizer.encode(text)
print(f"Number of tokens: {len(tokens)}")

Number of tokens: 4


In [ ]:
# tokenized_datasets.save_to_disk("th-en-tokenized")
# !zip -r th-en-tokenized.zip th-en-tokenized

In [ ]:
tokenized_datasets['train']['input_ids'][1][:10]

[12, 38724, 175, 27180, 13605, 10869, 12, 38724, 12, 6545]

In [ ]:
tokenizer.convert_ids_to_tokens(tokenized_datasets['train']['input_ids'][1])[:10]

['▁',
 'ปาเลสไตน์',
 'ใน',
 'โอลิมปิก',
 'ฤดูร้อน',
 '▁2004',
 '▁',
 'ปาเลสไตน์',
 '▁',
 'เข้าร่วม']

# Fine tuning

In [ ]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62307, 512, padding_idx=62306)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62307, 512, padding_idx=62306)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

1.same size components     
2.tensor    
3.create decoder_input

In [ ]:
import torch

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([{
    'input_ids': tokenized_datasets['train'][i]['input_ids'],
    'attention_mask': tokenized_datasets['train'][i]['attention_mask'],
    'labels': tokenized_datasets['train'][i]['labels'],
} for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch['input_ids'][0]

tensor([   12, 38724,   175, 27180, 13605, 10869,    12, 38724,    12,  6545,
        16278, 11858, 27180, 13605, 10798,  3017,  1540,     2,  2840,     2,
        10869,    77,  1197,     2,  2840,     2,  1723, 16363,    80,  1288,
         1923,  1442, 11816, 14392,   869, 23353,  1589,  1795,  3627,  1557,
           12, 11059,  3968, 11112,  6755,     2,  2840,     2,  1723, 16363,
            0, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306,
        62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306,
        62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306,
        62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306,
        62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306,
        62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306,
        62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306,
        62306, 62306, 62306, 62306, 62306, 62306, 62306, 62306, 

In [ ]:
tokenizer.convert_ids_to_tokens(batch['input_ids'][0])

['▁',
 'ปาเลสไตน์',
 'ใน',
 'โอลิมปิก',
 'ฤดูร้อน',
 '▁2004',
 '▁',
 'ปาเลสไตน์',
 '▁',
 'เข้าร่วม',
 'แข่งขัน',
 'กีฬา',
 'โอลิมปิก',
 'ฤดูร้อน',
 'ครั้งที่',
 '▁28',
 '▁ค',
 '.',
 'ศ',
 '.',
 '▁2004',
 '▁(',
 'พ',
 '.',
 'ศ',
 '.',
 '▁25',
 '47',
 ')',
 '▁ณ',
 '▁กรุง',
 'เอ',
 'เธ',
 'นส์',
 '▁ประเทศ',
 'กรี',
 'ซ',
 'ระหว่าง',
 'วันที่',
 '▁13',
 '▁',
 '–',
 '▁29',
 '▁สิงหาคม',
 '▁พ',
 '.',
 'ศ',
 '.',
 '▁25',
 '47',
 '</s>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>',
 '<pad>

# Config model

In [ ]:
from datasets import load_metric
metric = load_metric("sacrebleu")

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

In [ ]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

# Train model

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    output_dir="2opus-mt-th-en-finetuned-en-to-th",  #folder name save results
    eval_strategy="steps",       eval_steps=500,    #display evaluation each step=batch (show batch 500) validation
    save_strategy="steps",       save_steps=1000,   #save model at step=1000
    learning_rate=2e-5,                             #learning rate
    per_device_train_batch_size=10,                  #number of batch per device for training
    per_device_eval_batch_size=2,                   #number of batch per device for validation
    weight_decay=0.01,                              #optimization AdamW
    save_total_limit=3,                             #limit save model if > 3 it delete old checkpoint and more new
    num_train_epochs=1,                             #number of epoch of training
    predict_with_generate=True,                     #allow generate new messages
    fp16=False,                                     #for GPU(Ture)
    push_to_hub=True,                               #pull hugging face
    report_to="none",                               #display website
    logging_strategy="steps",                       #display loss traing
    logging_steps=500                               #display loss each 500 step for training
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

<ipython-input-65-1468824c6e77>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.evaluate(max_length=512) # before training

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu
500,0.872700,0.854458,0.175596
1000,0.855800,0.832442,0.183032
1500,0.836300,0.819124,0.184212
2000,0.847300,0.808982,0.182925
2500,0.829700,0.803403,0.187563


TrainOutput(global_step=2617, training_loss=0.8480884346471836, metrics={'train_runtime': 3968.2948, 'train_samples_per_second': 6.593, 'train_steps_per_second': 0.659, 'total_flos': 2131903128010752.0, 'train_loss': 0.8480884346471836, 'epoch': 1.0})

In [ ]:
trainer.evaluate(max_length=512)

{'eval_loss': 0.8027006983757019,
 'eval_bleu': 9.82228053165611,
 'eval_runtime': 2192.3303,
 'eval_samples_per_second': 1.731,
 'eval_steps_per_second': 0.866,
 'epoch': 1.0}

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete") # อัพขึ้น hugging face

CommitInfo(commit_url='https://huggingface.co/Dinonon/2opus-mt-th-en-finetuned-en-to-th/commit/f170ec8d2d657ae63bf6204a77ff08e5d75d7d15', commit_message='Training complete', commit_description='', oid='f170ec8d2d657ae63bf6204a77ff08e5d75d7d15', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dinonon/2opus-mt-th-en-finetuned-en-to-th', endpoint='https://huggingface.co', repo_type='model', repo_id='Dinonon/2opus-mt-th-en-finetuned-en-to-th'), pr_revision=None, pr_num=None)

# Using

In [ ]:
from transformers import pipeline
model_checkpoint = "Dinonon/2opus-mt-th-en-finetuned-en-to-th"
translator = pipeline("translation", model=model_checkpoint)

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/875 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/810k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.53M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [ ]:
translator("ฉันรักเธอ")

[{'translation_text': 'I do.'}]